In [1]:
from tensorflow.keras.optimizers import Adam
from keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint
from model.core import RUnet_model
from utils.functions import r_score, mse_nonzero, data_split
import numpy as np

# Build and compile model

In [2]:
# build a DL model for first two levels
level1 = 1
level2 = 2

dl_model = RUnet_model(level1, level2)
opt = Adam(lr=1e-5) 

dl_model.compile(optimizer=opt, loss=mse_nonzero, metrics=[r_score, mse_nonzero])
dl_model.info()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 model_input (InputLayer)       [(None, 48, 376, 10  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda)                (None, 48, 376, 10)  0           ['model_input[0][0]']            
                                                                                                  
 Block1_Conv1 (Conv2D)          (None, 48, 376, 128  11648       ['lambda[0][0]']                 
                                )                                                                 
                                                                                              

D:\anaconda\envs\tf3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Load data sets using data generator

In [3]:
from utils.data_generator import data_generator
import glob

In [15]:
# phase-1 training using preprocessed B-SOSE data sets
xfiles = np.array(sorted(glob.glob('example_data\phase1_example_data\X/X*.npz')))
yfiles = np.array(sorted(glob.glob('example_data\phase1_example_data\Y/Y*.npz')))

print(xfiles, yfiles)

['example_data\\phase1_example_data\\X\\X_000.npz'
 'example_data\\phase1_example_data\\X\\X_001.npz'
 'example_data\\phase1_example_data\\X\\X_002.npz'
 'example_data\\phase1_example_data\\X\\X_003.npz'
 'example_data\\phase1_example_data\\X\\X_004.npz'
 'example_data\\phase1_example_data\\X\\X_1030.npz'
 'example_data\\phase1_example_data\\X\\X_1031.npz'
 'example_data\\phase1_example_data\\X\\X_1032.npz'
 'example_data\\phase1_example_data\\X\\X_1033.npz'
 'example_data\\phase1_example_data\\X\\X_1034.npz'] ['example_data\\phase1_example_data\\Y\\Y_000.npz'
 'example_data\\phase1_example_data\\Y\\Y_001.npz'
 'example_data\\phase1_example_data\\Y\\Y_002.npz'
 'example_data\\phase1_example_data\\Y\\Y_003.npz'
 'example_data\\phase1_example_data\\Y\\Y_004.npz'
 'example_data\\phase1_example_data\\Y\\Y_1030.npz'
 'example_data\\phase1_example_data\\Y\\Y_1031.npz'
 'example_data\\phase1_example_data\\Y\\Y_1032.npz'
 'example_data\\phase1_example_data\\Y\\Y_1033.npz'
 'example_data\\phase

In [16]:
# split data set into training, valid, testing sets
xtrain, ytrain, xvalid, yvalid, xtest, ytest = data_split(xfiles, yfiles, 0.7225, 0.85, maskname='sample_train_valid_test_mask.npz')
print(xtrain, ytrain)
print(xvalid, yvalid)
print(xtest, ytest)

[4 5 2 6 1 9 8]
['example_data\\phase1_example_data\\X\\X_004.npz'
 'example_data\\phase1_example_data\\X\\X_1030.npz'
 'example_data\\phase1_example_data\\X\\X_002.npz'
 'example_data\\phase1_example_data\\X\\X_1031.npz'
 'example_data\\phase1_example_data\\X\\X_001.npz'
 'example_data\\phase1_example_data\\X\\X_1034.npz'
 'example_data\\phase1_example_data\\X\\X_1033.npz'] ['example_data\\phase1_example_data\\Y\\Y_004.npz'
 'example_data\\phase1_example_data\\Y\\Y_1030.npz'
 'example_data\\phase1_example_data\\Y\\Y_002.npz'
 'example_data\\phase1_example_data\\Y\\Y_1031.npz'
 'example_data\\phase1_example_data\\Y\\Y_001.npz'
 'example_data\\phase1_example_data\\Y\\Y_1034.npz'
 'example_data\\phase1_example_data\\Y\\Y_1033.npz']
['example_data\\phase1_example_data\\X\\X_000.npz'] ['example_data\\phase1_example_data\\Y\\Y_000.npz']
['example_data\\phase1_example_data\\X\\X_1032.npz'
 'example_data\\phase1_example_data\\X\\X_003.npz'] ['example_data\\phase1_example_data\\Y\\Y_1032.npz'


In [17]:
train_generator = data_generator(xtrain, ytrain, level1, level2, batch_size=5)
valid_generator = data_generator(xvalid, yvalid, level1, level2, batch_size=5)

# Set up the training

In [18]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger

In [26]:
csv_logger = CSVLogger( 'sample_log.csv' , append=True, separator=';')
earlystopper = EarlyStopping(patience=20, verbose=1)
checkpointer = ModelCheckpoint('checkpt.h5', verbose=1, save_best_only=True)

In [ ]:
### For phase-2 training, need to load model weights from phase-1:
# dl_model.load_weights('path/to/phase1/model/weights')


dl_model.train(train_generator,
                    validation_data=valid_generator, epochs=250,
                    callbacks=[earlystopper, checkpointer, csv_logger])

# Save the model weights

In [ ]:
dl_model.save_model('sample_model.h5')

# Load model weights and predict DIC

In [13]:
from utils.functions import process_x, add_buffer

dl_model.load_weights('path/to/pretrained/model/weights')

for x in xtest:
    xdata = np.load(x)['x']
    xdata = xdata[np.newaxis, :, :, :]  # fill the batch dimension, 1, 10, 56, 360
    xdata = np.moveaxis(xdata, 1, -1)   # rotate the dimension of the input vector to match the model: 1, 56, 360, 10
    xdata = process_x(xdata)            # apply scaling factors
    xdata = add_buffer(xdata)           # add buffer domain
    pred = dl_model.predict(xdata)           
    pred = add_buffer(pred, direction=-1)[0]   # remove buffer domain
    filename = 'path/to/output/prediction_' + x.split('_')[-1]
    print(" >>>>>>>>> Saving: ", filename)
    np.save(filename, pred)

sample_data/phase1_sample_data/X\X_001.npz
X\X_001.npz
sample_data/phase1_sample_data/X\X_003.npz
X\X_003.npz
sample_data/phase1_sample_data/X\X_007.npz
X\X_007.npz
